In [2]:

import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt


In [5]:
training = pd.read_csv('./data/train_new_version.csv')
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


    Survived: результат выживания (0 = нет; 1 = да)
    Pclass: социально-экономический класс (1 = высший класс; 2 = средний класс; 3 = низший класс)
    Name: Имя пассажира
    Sex: Пол пассажира
    Age: возраст пассажира (некоторые записи содержат NaN).
    SibSp: количество братьев и сестер и супругов пассажира на борту
    Parch: количество родителей и детей пассажира на борту
    Ticket: номер билета пассажира.
    Fare: тариф оплачивается пассажиром.
    Cabin: Номер кабины пассажира (некоторые записи содержат NaN)
    Embarked: Порт посадки пассажира (C = Шербур; Q = Квинстаун; S = Саутгемптон)

In [6]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df_num = training[['Age','SibSp','Parch','Fare']]
df_cat = training[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [8]:
# сравним зависимость выживаемости от возраста, SibSp, Parch и Fare
pd.pivot_table(training, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

,Age,Fare,Parch,SibSp
Survived,,,,
0,30.626179,22.117887,0.329690,0.553734
1,28.343690,48.395408,0.464912,0.473684


##  Feature Engineering 

### 2) Билеты - влияют ли разные типы билетов на выживаемость?

### 3) Имеет ли отношение титул человека к выживаемости?

Добавим новый признак - количество кабин

In [9]:
training['cabin_multiple'] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
training['cabin_multiple'].value_counts()

cabin_multiple
0    687
1    180
2     16
3      6
4      2
Name: count, dtype: int64

In [10]:
pd.pivot_table(training, index = 'Survived', columns = 'cabin_multiple', values = 'Ticket' ,aggfunc ='count')

cabin_multiple,0,1,2,3,4
Survived,,,,,
0,481.0,58.0,7.0,3.0,NaN
1,206.0,122.0,9.0,3.0,2.0


In [11]:
# создает категории на основе буквы каюты (n означает ноль)
# в этом случае мы будем рассматривать нулевые значения как свою собственную категорию
training['cabin_adv'] = training.Cabin.apply(lambda x: str(x)[0])
training['cabin_adv'].unique()

array(['n', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [12]:
# сравнение выживаемости по каютам
print(training.cabin_adv.value_counts())
pd.pivot_table(training,index='Survived',columns='cabin_adv', values = 'Name', aggfunc='count')

cabin_adv
n    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: count, dtype: int64


cabin_adv,A,B,C,D,E,F,G,T,n
Survived,,,,,,,,,
0,8.0,12.0,24.0,8.0,8.0,5.0,2.0,1.0,481.0
1,7.0,35.0,35.0,25.0,24.0,8.0,2.0,NaN,206.0


In [13]:
# добавим два признака для билета
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
training[['numeric_ticket',"Ticket", 'ticket_letters']].head()

,numeric_ticket,Ticket,ticket_letters
0,0,A/5 21171,a5
1,0,PC 17599,pc
2,0,STON/O2. 3101282,stono2
3,1,113803,0
4,1,373450,0


In [14]:
training['numeric_ticket'].value_counts()

numeric_ticket
1    661
0    230
Name: count, dtype: int64

In [15]:
# разница в числовых и нечисловых билетах в выживаемости
pd.pivot_table(training,index='Survived',columns='numeric_ticket', values = 'Ticket', aggfunc='count')

numeric_ticket,0,1
Survived,,
0,142,407
1,88,254


In [16]:
# получим приставки к имени
training['name_title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
training['name_title'].head()

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: name_title, dtype: object

## Data Preprocessing

In [17]:
# заполним пропуски для непрерывных данных 
training.Age = training.Age.fillna(training.Age.median())
training.Fare = training.Fare.fillna(training.Fare.median())

# удаляем столбец с "высадкой". Только 2 случая в выборке.
training.dropna(subset=['Embarked'],inplace = True)

# преобразуем "тариф" в категорию для pd.get_dummies ()
training.Pclass = training.Pclass.astype(str)

# созданы фиктивные переменные из категорий (также можно использовать OneHotEncoder)
all_dummies = pd.get_dummies(training[['Pclass','Sex','Age','SibSp','Parch','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title']])

In [18]:
# масштабирование данных
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[['Age','SibSp','Parch']]= scale.fit_transform(all_dummies_scaled[['Age','SibSp','Parch']])
all_dummies_scaled

X_train_scaled = all_dummies_scaled

y_train = training.Survived




## Построение и сравнение моделей 

- Наивный байесовский классификатор
- Логистическая регрессия 
- Дерево решений 
- K Ближайших соседей 
- Случайный лес
- **SVM с RBF**
- Xgboost

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [20]:
# Будем использовать наивный байес в качестве baseline 
gnb = GaussianNB()
cv = cross_val_score(gnb,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.66853933 0.68539326 0.75842697 0.75280899 0.72881356]
0.7187964197295752


In [21]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.8258427  0.81460674 0.82022472 0.81460674 0.85310734]
0.8256776487018346


In [22]:
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.76966292 0.75280899 0.80337079 0.75842697 0.75706215]
0.7682663619628007


In [23]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.80898876 0.80898876 0.83146067 0.80337079 0.85310734]
0.8211832666793628


In [24]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.7752809  0.79213483 0.80337079 0.76966292 0.78531073]
0.7851520345331048


In [25]:
svc = SVC(probability = True)
cv = cross_val_score(svc,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.83707865 0.82022472 0.8258427  0.80337079 0.86440678]
0.8301847267187202


In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state =1)
cv = cross_val_score(xgb,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

[0.79775281 0.83146067 0.79775281 0.80898876 0.83050847]
0.8132927061512092


## Итоги

|Model|Score|
|-----|--------|
|Naive Bayes| 72.6%|
|Logistic Regression| 82.1%|
|Decision Tree| 77.6%|
|K Nearest Neighbor| 80.5%|
|Random Forest| 80.6%|
|Support Vector Classifier| 83.2%|
|Xtreme Gradient Boosting| 81.8%|
